This notebook is a follow up to a post I made on the forums. It's intended to be a minimalist example of how to evolve a finite state machine (FSM) to play Roshambo. I would have wrapped it up much sooner, but I got distracted playing with it.

# The Forum Post
I was describing the contest to someone recently and I brought up the "Iocaine" agent with references to the poison wine scene in "The Princess Bride." It's a useful metaphor to help answer: "why is this even interesting?"

The first part of my agent is much like what has been described many times. It has a big bank of prediction filters and a voting scheme to boil it all down. Based on the history of the match so far, it's calculating which action would do best on the next round. It's optimized just for that next round - pure offense. But, as in the wine scene, that's "just getting started." The other guy has the same historical information, and has presumably just made an equivalent calculation.

So, that predicted best action goes to the second part of my agent which treats it as merely a suggestion. (I'm calling the following the "survivor bias algorithm, (SBA)") The SBA has four tactics to choose from: take the predicted best action, the action that would beat it, the action that would lose to it, or disregard it entirely and pick a random action. The tactic is chosen by a finite state machine (FSM). Each state in the FSM has one tactic (one of the four), and three possible conditions: the previous round was a win, loss, or tie. So each state can be represented by four values: the tactic to use and the three new states to transition to based on the previous round's outcome. The FSM, an Nx4 array, is generated by a genetic algorithm, scored against a benchmark of agents I've written, pilfered, or evolved.

The SBA has some nice properties. In local testing, it wins more matches than just using the predicted best action would. Also, it's modular. I can evolve a FSM using a fast version of the filter bank, but swap in another version later. Or I can swap in a too-slow-to evolve-with published agent, eg. Iocaine. (The result with Iocaine is around the same strength as with my own filter - better against some opponents, not as good against others. It does comparably well on the leaderboard. Not too surprisingly, SBA plus Iocaine wins 100 percent of its matches against Iocaine alone - the "evil twin effect.")

There're a lot of things the SBA is not doing. It's not just taking the predicted actions and throwing in random moves. As is the norm with evolved solutions, I can trace the logic behind decisions the SBA makes, but see no plan or pattern behind them. Presumably, that's the point.

# A quick note about Numba.
This notebook makes extensive use of Numba, a just-in-time (JIT) compiler. I've found Numba to be both a blessing and a curse. My code runs two orders of magnitude faster, but plenty of perfectly good Python code doesn't play well with Numba. C-like structure seems to work well for me. YMMV.


# And on to the example

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
from datetime import timedelta
from numba import njit # A just-in-time (JIT) compiler

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

First, we'll need a simple stand-in for the prediction filter. Always predicting that rock would be the best choice isn't the greatest prediction, but we'll see what the GA can make of it.

In [ ]:

@njit()
def box_of_rocks_predictor(step, my_last_action, lastOpponentAction):
    """ 
    My prediction filter always predicts that playing rock will surely win this time.
    It is often in error, never in doubt.
    """
    return 0

Now, some fast opponents, slightly modified from the versions here: https://www.kaggle.com/ihelon/rock-paper-scissors-agents-comparison

In [ ]:
@njit()
def get_score(friend_move, foe_move):
    """ utility function for reactionary and counter_reactionary """
    if (friend_move == foe_move):
        return(1) #ties+= 1
    else:
        wining_move = (foe_move + 1) % 3
        if (friend_move == wining_move):
            return(2) #wins += 1
        else:
            return(0) #losses += 1
    print('BADNESS in get_score', friend_move, foe_move)
    return(-1) #los

@njit()
def mirror_agent(step, my_last_action, lastOpponentAction):
    if step == 0:
        action = int(np.random.randint(3))
    else:
        action = lastOpponentAction
    return (int(action))
    
@njit()
def hit_the_last_own_action(step, my_last_action, lastOpponentAction):
    if (step == 0):
        return(int(np.random.randint(3)))
    action = (my_last_action + 1) % 3
    return action
    
@njit()
def rock(step, my_last_action, lastOpponentAction):
    return 0
   
@njit()
def paper(step, my_last_action, lastOpponentAction):
    return 1 
  
@njit()
def scissors(step, my_last_action, lastOpponentAction):
    return 2
    
@njit()
def reactionary(step, my_last_action, lastOpponentAction):
    action = my_last_action
    if (step == 0):
        return(int(np.random.randint(3)))
    elif get_score(my_last_action, lastOpponentAction) <= 1:
        action = (lastOpponentAction + 1) % 3
    return action
    
@njit()
def counter_reactionary(step, my_last_action, lastOpponentAction):
    action = my_last_action
    if (step == 0):
        return(int(np.random.randint(3)))
    elif get_score(my_last_action, lastOpponentAction) == 1:
        action = (lastOpponentAction + 2) % 3
    else: action = (lastOpponentAction + 1) % 3
    return action

@njit()
def switching0(step, my_last_action, lastOpponentAction):
    q = step // 200
    if (q == 0):    return (np.random.randint(3) )
    elif (q == 1):  return( reactionary(step, my_last_action, lastOpponentAction) )
    elif (q == 2):  return( counter_reactionary(step, my_last_action, lastOpponentAction) )
    elif (q == 3):  return( counter_reactionary(step, my_last_action, lastOpponentAction) )
    elif (q == 4):  return( counter_reactionary(step, my_last_action, lastOpponentAction) )   
    return (np.random.randint(3))

@njit()
def switching1(step, my_last_action, lastOpponentAction):
    q = step // 200
    if (q == 0):    return (np.random.randint(3) )
    elif (q == 1):  return(np.random.randint(3))
    elif (q == 2):  return(np.random.randint(3))
    elif (q == 3):  return( 1 )
    elif (q == 4):  return( 0 )   
    return (np.random.randint(3))

@njit()
def switching2(step, my_last_action, lastOpponentAction):
    q = step // 200
    if (q == 0):    return (np.random.randint(3) )
    elif (q == 1):  return( mirror_agent(step, my_last_action, lastOpponentAction) )
    elif (q == 2):  return( mirror_agent(step, my_last_action, lastOpponentAction) )
    elif (q == 3):  return( hit_the_last_own_action(step, my_last_action, lastOpponentAction) )
    elif (q == 4):  return( hit_the_last_own_action(step, my_last_action, lastOpponentAction) )  
    return (np.random.randint(3))

@njit()
def switching3(step, my_last_action, lastOpponentAction):
    q = step // 200
    if (q == 0):    return (np.random.randint(3) )
    elif (q == 1):  return( q )
    elif (q == 2):  return( q )
    elif (q == 3):  return( hit_the_last_own_action(step, my_last_action, lastOpponentAction) )
    elif (q == 4):  return( hit_the_last_own_action(step, my_last_action, lastOpponentAction) )  
    return (np.random.randint(3))

@njit()
def switching4(step, my_last_action, lastOpponentAction):
    if (step < 800):    return (np.random.randint(3) )
    return (step % 3)


Finite state machine (FSM) code

In [ ]:
@njit()
def FSM_agentX(step, prev_friend_act, prev_foe_act, state, filter_move, state_choices, action_choices):
    """ 
    Choose 1 move using finite state machine (FSM) 
    - Test the condition (win, lose, tie)
    - Move to the new state
    - Take the strategy (a) from that state
    - Translate the strategy to an action
    - Return the new state and the action
    """
    # num_actions == 3
    holdoff = 0 # spin at state zero until past this step
# =============================================================================
#     4 possible actions:
#       filter_move
#       counter to filter_move
#       counter-counter to filter_move
#       random
#     3 possible conditions: win, lose, tie
# =============================================================================
    if step == 0:
        condition = 2 # tie
        if (state != 0):
            print("FSM_agentX, starting state =", state)
    if step <= holdoff:
        condition = 2 # tie
        #    action = np.int32(np.random.randint(3))
        state = state_choices[state, condition] # get next state
        #a = np.int32(action_choices[state]) # get action to play
        action = np.int32(np.random.randint(3))
        state = 0 # remain at state 0
        #if (showit): print('FSM_agentX() step 0. action', action, 'state', state)
        return (state, action)
    
    # find new state based on condion (win, lose, tie)
    if (prev_friend_act == prev_foe_act): condition = 2 # tie
    elif (prev_friend_act == (prev_foe_act +1) % 3): condition = 0 # win
    else: condition = 1 # lose
    
    #if (condition != 0): #only change state on a loss or tie
    state = state_choices[state, condition] # get next state
    a = np.int32(action_choices[state]) # get strategy to play
        
    if (a < 3): # pick action relative to filter_move
        action = (filter_move + a) % 3
    else: action = int(np.random.randint(3))
    return (state, action)       


> Utilities to play one match against a designated opponent. This is where we set which predictor we'll be using to evolve with.

In [ ]:

@njit()
def FSM_move(s, prev_friend_move, prev_foe_move, Astate, states_A, actions_A):
    """ 
    Choose the next move, using the prediction filter and the FSM
    s         -> step
    Astate    -> current state in the FSM
    states_A  -> part of FSM
    actions_A -> the other part of the FSM
    """
    # THIS IS WHERE THE PREDICTOR GOES. YOUR MESSAGE COULD BE HERE.
    filter_prediction = box_of_rocks_predictor(s, prev_friend_move, prev_foe_move)
    # filter_prediction is passed to the FSM which treats it as merely a suggestion. The predictor needs to be able to
    # make predictions whether its suggested actions are used or not.
    
    Astate, friend_move = FSM_agentX(s, prev_friend_move, prev_foe_move, Astate, filter_prediction, states_A, actions_A)
    # friend_move is the action actually played.
    return(Astate, friend_move)
         
         
@njit()
def run_FSM_bench(states_A, actions_A, opponent):
    """ 
    Play one match (1000 rounds) against this opponent, and return the outcome.
    
    If you want to add or change the opponents to evolve against, adjust that here.
    Also update full_benchmarking() if the number of opponents changes.
    Opponents need to be compatible with Numba and fairly fast.
    """
    trials = 1000
    freezing = np.array([34, 40, 44, 48, 51, 54, 57, 59, 61]) # 95% safety if switching to random moves
    use_mercy_rule = False
    prev_foe_move = -1
    prev_friend_move = -1
    wins = 0
    losses = 0
    ties = 0
    Astate = 0 # We start a match at state zero, by convention.   
    for s in range(trials):
        # friend move 
        Astate, friend_move = FSM_move(s, prev_friend_move, prev_foe_move, Astate, states_A, actions_A)
         
        # opponent (foe) move
        if   (opponent == 0):  foe_move = rock                   (s, prev_foe_move, prev_friend_move)
        elif (opponent == 1):  foe_move = paper                  (s, prev_foe_move, prev_friend_move)
        elif (opponent == 2):  foe_move = scissors               (s, prev_foe_move, prev_friend_move)           
        elif (opponent == 3):  foe_move = hit_the_last_own_action(s, prev_foe_move, prev_friend_move)
        elif (opponent == 4):  foe_move = mirror_agent           (s, prev_foe_move, prev_friend_move)
        elif (opponent == 5):  foe_move = reactionary            (s, prev_foe_move, prev_friend_move)
        elif (opponent == 6):  foe_move = counter_reactionary    (s, prev_foe_move, prev_friend_move)
        elif (opponent == 7):  foe_move = switching0             (s, prev_foe_move, prev_friend_move)
        elif (opponent == 8):  foe_move = switching1             (s, prev_foe_move, prev_friend_move)
        elif (opponent == 9):  foe_move = switching2             (s, prev_foe_move, prev_friend_move)
        elif (opponent == 10): foe_move = switching3             (s, prev_foe_move, prev_friend_move)
        elif (opponent == 11): foe_move = switching4             (s, prev_foe_move, prev_friend_move)
        else: print('bad foe in fullbench')
        
        # score this round
        if (friend_move == foe_move):
            ties+= 1
        else:
            wining_move = (foe_move + 1) % 3
            if (friend_move == wining_move):
                wins += 1
            else:
                losses += 1
        prev_foe_move = foe_move
        prev_friend_move = friend_move
        if (use_mercy_rule and (s > 100)): # early stopping of the match when it's a blowout
                temp = (1000 - s) // 100
                freezingPoint = freezing[temp]
                if ((wins - losses > freezingPoint) or (losses - wins > freezingPoint)): return([wins, losses, ties])
    #print('opponent', opponent, "wins. losses", wins, losses, "delta", (wins - losses))
    return([wins, losses, ties])


Utilities to run multiple matches against the fast opponents

In [ ]:
@njit()
def run_many_benches(state_choices, action_choices, matches, opponent):
    """ Run several matches against this opponent """
    W = np.int32(0)
    L = np.int32(0)
    results =  np.zeros(matches, np.int32) 
    for m in range(matches):
        wins, losses, ties = run_FSM_bench(state_choices, action_choices, opponent)
        results[m] = wins - losses
        W += wins
        L += losses
    W = W // matches
    L = L // matches
    #print("run_many_benches W,L", W, L)
    return(results)

@njit()
def score_matches_thresh(results):
    """ 
    Return the score across the results of multiple matches.
    Assign a match a win if the margin of victory >= 20, etc. 
    """
    wins = np.sum(results >= 20)
    losses = np.sum(results <= -20)
    #ties = np.sum(matches == 0)
    score = np.int32((100 * (wins - losses)) // len(results))
    return(score)

@njit()
def full_benchmarking(state_choices, action_choices, matches, showit = False):
    """ 
    Run many matches against all 12 opponents and return a score for the evolver.
    """
    b = 12 # There are 12 opponents to test against. (Keep compatible with run_FSM_bench())
    scores = np.zeros(b, np.int32) 
    beauty = np.zeros(b, np.int32) 
    for i in range(b):
        if (i < 3): reps = matches // 4
        else: reps = matches
        results = run_many_benches(state_choices, action_choices, reps, i)
        scores[i] = score_matches_thresh(results)
        beauty[i] = np.mean(results)
    if (showit): 
        print("avg. winning percentage", scores)
        print("avg. margin of victory ", beauty)
    return(np.mean(scores), np.min(beauty))


The GA

In [ ]:
# =============================================================================
#        INDIVIDUAL
#        INDIVIDUAL
#        INDIVIDUAL
# =============================================================================

class INDIVIDUAL():
    
    """
    This class holds an individual for a GA.
    It represents a finite state machine to play Roshambo.
    """
    def __init__(self):
        self.N = 200 # number of states
        self.action_choices = np.zeros((self.N), np.int32)   # coded state table (0-4)
        self.state_choices = np.zeros((self.N, 3), np.int32) # coded state table (0-N)
        
 
@njit() 
def init_individual(N):
    """
    Create new state_choices[] and action_choices[]
    action_choices[] 0 <= a < 6
    state_choices[] 0 <= s <= N
    """
    num_actions = 4
    state_choices = np.zeros((N, 3), np.int32) 
    action_choices = np.zeros(N, np.int32) # coded state table (0-4)
    for k in range(N):
        action_choices[k] = int(np.random.randint(num_actions)) 
        for i in range(3):
            state_choices[k, i] = int(np.random.randint(N))

    return(state_choices, action_choices)  

@njit()  
def procreate_individual(N, par1_states, par1_actions, par2_states, par2_actions):
    """
    Crosover and mutation
    action_choices[] 0 <= a < num_actions
    state_choices [] 0 <= s < N
    """
    #print("procreate_individual start")
    num_actions = 4
    P_mutate = 3
    child_states = np.zeros((N, 3), np.int32) 
    child_actions = np.zeros(N, np.int32) # coded state table (0-4)
    # perform two-point crossover, combined with mutation
    c1 = np.random.randint(N)
    c2 = np.random.randint(N)
    if (c1 > c2):
       c1, c2 = c2, c1
    for k in range(N):
        if ((k <= c1) or (k > c2)):
            child_actions[k] = par1_actions[k]
            if (np.random.randint(100) < P_mutate): 
                child_actions[k] = np.random.randint(num_actions)
            for i in range(3):
                child_states[k, i] = par1_states[k, i]
                if (np.random.randint(100) < P_mutate):
                     child_states[k, i] = np.random.randint(N)
        else:
            child_actions[k] = par2_actions[k]
            if (np.random.randint(100) < P_mutate): 
                child_actions[k] = np.random.randint(num_actions) 
            for i in range(3):
                child_states[k, i] = par2_states[k, i]
                if (np.random.randint(100) < P_mutate):
                    child_states[k, i] = np.random.randint(N)
    #print("procreate_individual finished")
    return(child_states, child_actions)   

    
    
def init_ga(popsize):
    """ create a new population  """
    pop = []
    for p in range(popsize):
        F = INDIVIDUAL()
        F.state_choices, F.action_choices = init_individual(F.N)
        pop.append(F)
    print('new population formed')
    return(pop)

@njit()
def tourn(scores, chances):
    """ Select one parent using tournament selection """
    popsize = len(scores)
    best = np.random.randint(popsize)
    for p in range(chances):
        x = np.random.randint(popsize)
        if (scores[best] < scores[x]): best = x
    return(best)

def run_ga(pop, generations):
    matches = 200 # Number of matches between pairs of agents. (Set lower for faster/noisier evolving.)
    popsize = len(pop)
    scores = np.zeros(popsize, np.float32)
    age = np.zeros(popsize, np.int32)
    beauty_spot_scores = np.zeros(popsize, np.int32)
    print("starting GA")
    start_time = time.time()
    for g in range(generations): # for every generation
        ts = time.time()
        for p in range(popsize): # score every individual
            scores[p], beauty_spot_scores[p]  = full_benchmarking( pop[p].state_choices, pop[p].action_choices, matches)
            
        print('gen', g, "STATS (min, mean, max)", np.min(scores), np.mean(scores), np.max(scores), end = ' ')
        tsA = time.time()
        print('time for testing pop', timedelta(seconds = (time.time() - ts))) 
        best = np.argmax(scores)
        print('Performance of best individual')
        full_benchmarking( pop[best].state_choices, pop[best].action_choices, matches, True)
        # save the best solution found so far in two files
        np.save('state_choice', pop[best].state_choices)   
        np.save('action_choices', pop[best].action_choices) 
        for new_child in range(popsize):
            p = np.argmin(scores)   # kill off the weakest individual
            par1 = tourn(scores, 2)
            par2 = tourn(scores, 2)
            #par2 = tourn(beauty_spot_scores, 2) # alternative choice for second parent
            if (np.random.randint(100) < 50): # randomly swap parents
                par1, par2 = par2, par1
            # replace the weakest individual with the offspring of the two parents
            pop[p].state_choices, pop[p].action_choices =  procreate_individual(pop[par1].N, 
                                    pop[par1].state_choices, pop[par1].action_choices, 
                                    pop[par2].state_choices, pop[par2].action_choices)
            # score the new individual
            scores[p], beauty_spot_scores[p]  = full_benchmarking( pop[p].state_choices, pop[p].action_choices, matches)
            
            age[p] = g
        #print('gen', g, "STATS ", np.min(scores), np.mean(scores), np.max(scores), end = ' ')
        #print('gen', g, "BEAUTY", np.min(beauty_spot_scores), 
        #      np.mean(beauty_spot_scores), np.max(beauty_spot_scores))
        print(" (", np.sum(age == g), 'new). Time for this generation',timedelta(seconds = (time.time() - ts))) 
    # swap best into slot zero
    best = np.argmax(scores)
    pop[0], pop[best] = pop[best], pop[0] 
    print('FINISHED. run time', timedelta(seconds = (time.time() - start_time)))
    return(pop)

#pop  = init_ga(200)  
#pop = run_ga(pop, 20)    


Create a new population with 200 individuals

In [ ]:
pop  = init_ga(200)

Run the GA for 20 generations. It should take about 10 minutes. We may want to do this a couple times, to make sure the agent beats all the opponents 100 % of the time.

(Each generation takes ~ 30 seconds under the default settings. The population size, number of matches between agents, and the number of opponents all impact how long this will take. A population of 200, playing 200 matches against 12 opponents twice, entails running nearly a million Roshambo matches each generation.)

*OPPONENTS: rock, paper, scissors, hit_the_last_own_action, mirror_agent, reactionary, counter_reactionary, switching0-4*

In [ ]:
pop = run_ga(pop, 20)  

So, now we've established that we can evolve a FSM, with a stopped-clock prediction filter, capable of beating a particular set of not too savvy opponents. That certainly raises more questions than it answers. For now, I will answer just one of them: "how can we make this agent portable in case we want ot test it somewhere else?"

First we'll set up a template with the required wrapper for this contest. Then we'll cut and paste our new evolved FSM into it.
The best individual found is stored in pop[0] (The best FSM found is also stored in two files '/kaggle/working/state_choice.npy' and '/kaggle/working/action_choices.npy', so we could load from files if we wanted).

# The Template

In [ ]:
%%writefile your_agent.py
import numpy as np
import pickle
def box_of_rocks_predictor(step, my_last_action, lastOpponentAction):
    """ 
    My prediction filter always predicts that playing rock will surely win this time.
    It is often in error, never in doubt.
    """
    return 0

def FSM_move_No_Numba(s, prev_friend_move, prev_foe_move, Astate, states_A, actions_A):
    """ 
    Choose the next move, using the prediction filter and the FSM
    s         -> step
    Astate    -> current state in the FSM
    states_A  -> part of FSM
    actions_A -> the other part of the FSM
    """
    
    # THIS IS WHERE THE PREDICTOR GOES. YOUR MESSAGE COULD BE HERE.
    filter_prediction = box_of_rocks_predictor(s, prev_friend_move, prev_foe_move)
    # filter_prediction is passed to the FSM which treats it as merely a suggestion. The predictor needs to be able to
    # make predictions whether its suggested actions are used or not.
    
    Astate, friend_move = FSM_agentX(s, prev_friend_move, prev_foe_move, Astate, filter_prediction, states_A, actions_A)
    return(Astate, friend_move)

def FSM_agentX(step, prev_friend_act, prev_foe_act, state, filter_move, state_choices, action_choices):
    """ 
    Choose 1 move using finite state machine (FSM) 
    - Test the condition (win, lose, tie)
    - Move to the new state
    - Take the strategy (a) from that state
    - Translate the strategy to an action
    - Return the new state and the action
    """
    # num_actions == 3
    holdoff = 0 # spin at state zero until past this step
# =============================================================================
#     4 possible actions:
#       filter_move
#       counter to filter_move
#       counter-counter to filter_move
#       random
#     3 possible conditions: win, lose, tie
# =============================================================================
    if step == 0:
        condition = 2 # tie
        if (state != 0):
            print("FSM_agentX, starting state =", state)
    if step <= holdoff:
        condition = 2 # tie
        #    action = np.int32(np.random.randint(3))
        state = state_choices[state, condition] # get next state
        #a = np.int32(action_choices[state]) # get action to play
        action = np.int32(np.random.randint(3))
        state = 0 # remain at state 0
        #if (showit): print('FSM_agentX() step 0. action', action, 'state', state)
        return (state, action)
    
    # find new state based on condion (win, lose, tie)
    if (prev_friend_act == prev_foe_act): condition = 2 # tie
    elif (prev_friend_act == (prev_foe_act +1) % 3): condition = 0 # win
    else: condition = 1 # lose
    
    #if (condition != 0): #only change state on a loss or tie
    state = state_choices[state, condition] # get next state
    a = np.int32(action_choices[state]) # get strategy to play
        
    if (a < 3): # pick action relative to filter_move
        action = (filter_move + a) % 3
    else: action = int(np.random.randint(3))
    return (state, action)       

class INDIVIDUAL():
    
    """
    This class holds an individual for a GA.
    It represents a finite state machine to play Roshambo.
    """
    def __init__(self):
        self.N = 200 # number of states
        self.action_choices = np.zeros((self.N), np.int32)    # coded state table (0-4)
        self.state_choices  = np.zeros((self.N, 3), np.int32) # coded state table (0-N)

F =  INDIVIDUAL() 

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ CUT HERE ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# THE FOLOWING 2 LINES HOLD AN OLD EVOLVED FSM. REPLACE THEM WITH THE NEW ONES.
F.action_choices = pickle.loads(b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\xc8\x85q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00i4q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89B \x03\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00q\rtq\x0eb.')
F.state_choices  = pickle.loads(b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\xc8K\x03\x86q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00i4q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89B`\t\x00\x00\x0e\x00\x00\x00\xb8\x00\x00\x00K\x00\x00\x00\x1b\x00\x00\x00"\x00\x00\x00\x89\x00\x00\x00\x80\x00\x00\x00\x93\x00\x00\x00\x85\x00\x00\x00n\x00\x00\x00L\x00\x00\x00g\x00\x00\x00\x19\x00\x00\x00\t\x00\x00\x00\xa4\x00\x00\x00u\x00\x00\x00w\x00\x00\x00\x1a\x00\x00\x00~\x00\x00\x00\xbf\x00\x00\x00\xb4\x00\x00\x00\x17\x00\x00\x00G\x00\x00\x00s\x00\x00\x00f\x00\x00\x00[\x00\x00\x00g\x00\x00\x00K\x00\x00\x00\x96\x00\x00\x00B\x00\x00\x00w\x00\x00\x00|\x00\x00\x00>\x00\x00\x00n\x00\x00\x00T\x00\x00\x00\x08\x00\x00\x00\xb9\x00\x00\x00}\x00\x00\x00\xbe\x00\x00\x00k\x00\x00\x00l\x00\x00\x00\xa3\x00\x00\x00\xb1\x00\x00\x00h\x00\x00\x00\xb8\x00\x00\x00E\x00\x00\x00a\x00\x00\x00<\x00\x00\x00r\x00\x00\x00\x81\x00\x00\x00\x8a\x00\x00\x00T\x00\x00\x00\xbb\x00\x00\x00H\x00\x00\x00\x91\x00\x00\x00?\x00\x00\x00\xa1\x00\x00\x00\x87\x00\x00\x00k\x00\x00\x00\x03\x00\x00\x00\xa8\x00\x00\x00}\x00\x00\x00\x01\x00\x00\x00i\x00\x00\x00\xa0\x00\x00\x00;\x00\x00\x00\x06\x00\x00\x00\xc0\x00\x00\x00\xac\x00\x00\x00c\x00\x00\x00)\x00\x00\x00\x15\x00\x00\x00\x83\x00\x00\x00z\x00\x00\x00L\x00\x00\x00_\x00\x00\x00?\x00\x00\x00&\x00\x00\x00\x80\x00\x00\x00 \x00\x00\x00\x83\x00\x00\x00<\x00\x00\x00p\x00\x00\x003\x00\x00\x00+\x00\x00\x00\x89\x00\x00\x00%\x00\x00\x00&\x00\x00\x00D\x00\x00\x00\xbc\x00\x00\x00g\x00\x00\x00g\x00\x00\x00\xad\x00\x00\x00\x82\x00\x00\x00\x01\x00\x00\x00M\x00\x00\x00>\x00\x00\x00\x03\x00\x00\x00\x97\x00\x00\x00\x1b\x00\x00\x00\x00\x00\x00\x00R\x00\x00\x00\xaf\x00\x00\x00\x98\x00\x00\x00\x91\x00\x00\x00\'\x00\x00\x00\xa4\x00\x00\x00\x9a\x00\x00\x00\x98\x00\x00\x00H\x00\x00\x00*\x00\x00\x00x\x00\x00\x00\x0c\x00\x00\x00w\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x97\x00\x00\x00a\x00\x00\x00^\x00\x00\x00\x1a\x00\x00\x00\x16\x00\x00\x00R\x00\x00\x00\x18\x00\x00\x00,\x00\x00\x00\x93\x00\x00\x00\r\x00\x00\x00\x85\x00\x00\x00\x95\x00\x00\x00\x08\x00\x00\x00\x82\x00\x00\x00\xa0\x00\x00\x00\x80\x00\x00\x00\xb8\x00\x00\x00\x04\x00\x00\x00\x94\x00\x00\x00\x0e\x00\x00\x00\x8b\x00\x00\x00\xa4\x00\x00\x00R\x00\x00\x001\x00\x00\x003\x00\x00\x00\x93\x00\x00\x00\x03\x00\x00\x00\'\x00\x00\x00N\x00\x00\x00\x1a\x00\x00\x00v\x00\x00\x00\x93\x00\x00\x00\x1a\x00\x00\x00"\x00\x00\x00R\x00\x00\x00Q\x00\x00\x00B\x00\x00\x00i\x00\x00\x00a\x00\x00\x00\x85\x00\x00\x00,\x00\x00\x008\x00\x00\x00!\x00\x00\x00{\x00\x00\x00t\x00\x00\x00\xb0\x00\x00\x00S\x00\x00\x00\x0c\x00\x00\x00f\x00\x00\x00/\x00\x00\x00\xa1\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\xa5\x00\x00\x00\xb0\x00\x00\x00\x98\x00\x00\x00\x1c\x00\x00\x00\x95\x00\x00\x00\x9a\x00\x00\x00\x1d\x00\x00\x001\x00\x00\x00\xaf\x00\x00\x00~\x00\x00\x00Y\x00\x00\x00\xa0\x00\x00\x00v\x00\x00\x00\xc3\x00\x00\x00U\x00\x00\x00\x82\x00\x00\x00\x7f\x00\x00\x00\xa0\x00\x00\x00(\x00\x00\x00\x0b\x00\x00\x00.\x00\x00\x00\x97\x00\x00\x00^\x00\x00\x00\x15\x00\x00\x00P\x00\x00\x00c\x00\x00\x00\xbf\x00\x00\x00t\x00\x00\x00$\x00\x00\x00\x00\x00\x00\x00\r\x00\x00\x00\xa4\x00\x00\x00\x8c\x00\x00\x00b\x00\x00\x00\xa5\x00\x00\x00\xac\x00\x00\x00\x85\x00\x00\x00\xc1\x00\x00\x00\xb1\x00\x00\x00:\x00\x00\x00D\x00\x00\x00d\x00\x00\x00\x10\x00\x00\x00\xb9\x00\x00\x00\x07\x00\x00\x00"\x00\x00\x00\xbd\x00\x00\x00\xbb\x00\x00\x00\x17\x00\x00\x00\xb5\x00\x00\x008\x00\x00\x00\x8e\x00\x00\x00\xae\x00\x00\x00l\x00\x00\x00(\x00\x00\x00"\x00\x00\x00\x15\x00\x00\x00\x15\x00\x00\x00\x9e\x00\x00\x00\xa6\x00\x00\x00\xa5\x00\x00\x00\xaf\x00\x00\x00\x81\x00\x00\x00_\x00\x00\x00^\x00\x00\x00X\x00\x00\x00$\x00\x00\x00\x92\x00\x00\x00\xc6\x00\x00\x00U\x00\x00\x00\x9d\x00\x00\x00U\x00\x00\x00\x90\x00\x00\x00\xaf\x00\x00\x00p\x00\x00\x00k\x00\x00\x00.\x00\x00\x00W\x00\x00\x00\x81\x00\x00\x00D\x00\x00\x00\x1f\x00\x00\x00\xab\x00\x00\x00T\x00\x00\x00\xa3\x00\x00\x00\xaf\x00\x00\x00\xb2\x00\x00\x00\xb9\x00\x00\x00\x88\x00\x00\x00\xa5\x00\x00\x00\x86\x00\x00\x00b\x00\x00\x00\xc7\x00\x00\x00\xac\x00\x00\x00\xb7\x00\x00\x00X\x00\x00\x00\x0f\x00\x00\x00\x11\x00\x00\x00\x94\x00\x00\x00z\x00\x00\x00\x17\x00\x00\x00\x8a\x00\x00\x00e\x00\x00\x00\xbb\x00\x00\x00\x15\x00\x00\x00\x01\x00\x00\x00\x97\x00\x00\x00\x86\x00\x00\x00\x9a\x00\x00\x00a\x00\x00\x00c\x00\x00\x00\x90\x00\x00\x00c\x00\x00\x00!\x00\x00\x005\x00\x00\x00v\x00\x00\x00\x18\x00\x00\x00"\x00\x00\x00P\x00\x00\x006\x00\x00\x00[\x00\x00\x00P\x00\x00\x00\xc2\x00\x00\x00\xa5\x00\x00\x00\x1d\x00\x00\x00_\x00\x00\x00;\x00\x00\x00\xbe\x00\x00\x00\x12\x00\x00\x00\x98\x00\x00\x00c\x00\x00\x00\xbc\x00\x00\x00t\x00\x00\x00i\x00\x00\x00&\x00\x00\x00\x92\x00\x00\x00 \x00\x00\x00\x92\x00\x00\x00\t\x00\x00\x00[\x00\x00\x00\xb2\x00\x00\x00!\x00\x00\x00t\x00\x00\x00R\x00\x00\x00\x97\x00\x00\x00O\x00\x00\x00\x8b\x00\x00\x00 \x00\x00\x00Q\x00\x00\x002\x00\x00\x00=\x00\x00\x00m\x00\x00\x00\xa8\x00\x00\x00\x9c\x00\x00\x00h\x00\x00\x006\x00\x00\x00b\x00\x00\x005\x00\x00\x00\xaf\x00\x00\x00d\x00\x00\x00v\x00\x00\x00&\x00\x00\x00\x14\x00\x00\x00\xa6\x00\x00\x00\xb2\x00\x00\x00&\x00\x00\x00K\x00\x00\x00\x89\x00\x00\x00\x99\x00\x00\x00\x8f\x00\x00\x00#\x00\x00\x00\x87\x00\x00\x00l\x00\x00\x00B\x00\x00\x00\t\x00\x00\x00+\x00\x00\x00x\x00\x00\x009\x00\x00\x00h\x00\x00\x00,\x00\x00\x00\x1c\x00\x00\x00"\x00\x00\x00}\x00\x00\x00\x1e\x00\x00\x00\x93\x00\x00\x00\x84\x00\x00\x00*\x00\x00\x00\x89\x00\x00\x00\x18\x00\x00\x00\xa0\x00\x00\x00\x1e\x00\x00\x00\xa5\x00\x00\x00\x86\x00\x00\x00\xac\x00\x00\x00\x1c\x00\x00\x00\xa4\x00\x00\x00\xbb\x00\x00\x00\x87\x00\x00\x00!\x00\x00\x00_\x00\x00\x00\x8a\x00\x00\x00\x05\x00\x00\x00$\x00\x00\x00/\x00\x00\x00p\x00\x00\x00f\x00\x00\x00\xb7\x00\x00\x00P\x00\x00\x00)\x00\x00\x00[\x00\x00\x00\x02\x00\x00\x00c\x00\x00\x00=\x00\x00\x00G\x00\x00\x00>\x00\x00\x00X\x00\x00\x00<\x00\x00\x009\x00\x00\x00\x08\x00\x00\x00\xa8\x00\x00\x00b\x00\x00\x00\x95\x00\x00\x00\x8f\x00\x00\x00\xa2\x00\x00\x00\xad\x00\x00\x00\x9b\x00\x00\x00\x8f\x00\x00\x00`\x00\x00\x00{\x00\x00\x00\x9f\x00\x00\x009\x00\x00\x00\xb0\x00\x00\x00f\x00\x00\x00y\x00\x00\x009\x00\x00\x00\x88\x00\x00\x00s\x00\x00\x00\xa5\x00\x00\x00\x1b\x00\x00\x00f\x00\x00\x00\x9f\x00\x00\x00\xaf\x00\x00\x00\r\x00\x00\x00:\x00\x00\x00\xb4\x00\x00\x00"\x00\x00\x00\x7f\x00\x00\x00\xae\x00\x00\x00\xbd\x00\x00\x00s\x00\x00\x00L\x00\x00\x00\xae\x00\x00\x00\x13\x00\x00\x00\x96\x00\x00\x00\x06\x00\x00\x00k\x00\x00\x00\xaa\x00\x00\x00\xc3\x00\x00\x003\x00\x00\x00\xaa\x00\x00\x00\x00\x00\x00\x00\x8f\x00\x00\x00\xa5\x00\x00\x00J\x00\x00\x00*\x00\x00\x00\x85\x00\x00\x00\xb8\x00\x00\x00\\\x00\x00\x00L\x00\x00\x00v\x00\x00\x00]\x00\x00\x00l\x00\x00\x00p\x00\x00\x007\x00\x00\x00\x82\x00\x00\x00q\x00\x00\x00I\x00\x00\x00\xc3\x00\x00\x00@\x00\x00\x00X\x00\x00\x00&\x00\x00\x00R\x00\x00\x00c\x00\x00\x00\x0f\x00\x00\x00\x03\x00\x00\x00S\x00\x00\x00\xb6\x00\x00\x00a\x00\x00\x00\n\x00\x00\x00\x84\x00\x00\x00\x8f\x00\x00\x00\xc5\x00\x00\x007\x00\x00\x00\x16\x00\x00\x00J\x00\x00\x00\x14\x00\x00\x00\x9a\x00\x00\x00q\x00\x00\x00\x8d\x00\x00\x00\x1a\x00\x00\x00\xb5\x00\x00\x00)\x00\x00\x00\x19\x00\x00\x005\x00\x00\x00<\x00\x00\x00,\x00\x00\x00\x89\x00\x00\x005\x00\x00\x007\x00\x00\x00d\x00\x00\x00x\x00\x00\x005\x00\x00\x00^\x00\x00\x00x\x00\x00\x00\xa3\x00\x00\x006\x00\x00\x00\x8c\x00\x00\x00\x00\x00\x00\x00u\x00\x00\x00(\x00\x00\x00\xab\x00\x00\x00l\x00\x00\x00]\x00\x00\x00k\x00\x00\x00e\x00\x00\x00\xab\x00\x00\x00B\x00\x00\x00\xc5\x00\x00\x00E\x00\x00\x00\x81\x00\x00\x00\n\x00\x00\x00&\x00\x00\x00\x15\x00\x00\x00S\x00\x00\x006\x00\x00\x00a\x00\x00\x00\x01\x00\x00\x00R\x00\x00\x00*\x00\x00\x00\xb4\x00\x00\x00\x13\x00\x00\x00/\x00\x00\x00\xb4\x00\x00\x00u\x00\x00\x00\xbe\x00\x00\x00\x06\x00\x00\x00w\x00\x00\x00\x8c\x00\x00\x00\xc5\x00\x00\x00\x8d\x00\x00\x00\x13\x00\x00\x00.\x00\x00\x00\x0b\x00\x00\x00\x99\x00\x00\x00!\x00\x00\x00=\x00\x00\x00}\x00\x00\x00\x14\x00\x00\x00k\x00\x00\x00\xaf\x00\x00\x00E\x00\x00\x00\x8d\x00\x00\x00\x85\x00\x00\x00\x14\x00\x00\x00A\x00\x00\x00\x96\x00\x00\x00\x1a\x00\x00\x00|\x00\x00\x00\x98\x00\x00\x00r\x00\x00\x00*\x00\x00\x00D\x00\x00\x00~\x00\x00\x00{\x00\x00\x00F\x00\x00\x00)\x00\x00\x00\x06\x00\x00\x00c\x00\x00\x00y\x00\x00\x00H\x00\x00\x008\x00\x00\x001\x00\x00\x00\xbd\x00\x00\x00\x8a\x00\x00\x004\x00\x00\x00\x82\x00\x00\x00\xc4\x00\x00\x00\x1b\x00\x00\x00^\x00\x00\x00\xba\x00\x00\x00\xb5\x00\x00\x00\x12\x00\x00\x007\x00\x00\x00\xa5\x00\x00\x00j\x00\x00\x00\xa5\x00\x00\x00\x12\x00\x00\x00d\x00\x00\x00\x1a\x00\x00\x00\xa9\x00\x00\x00N\x00\x00\x00\xb8\x00\x00\x00\x86\x00\x00\x00&\x00\x00\x008\x00\x00\x00\x99\x00\x00\x00<\x00\x00\x00\x88\x00\x00\x00\x1d\x00\x00\x00`\x00\x00\x00.\x00\x00\x00\xa1\x00\x00\x00\xac\x00\x00\x00\x06\x00\x00\x00\x03\x00\x00\x00?\x00\x00\x00&\x00\x00\x00\xbe\x00\x00\x00|\x00\x00\x00\x9e\x00\x00\x00s\x00\x00\x00\xc3\x00\x00\x00"\x00\x00\x00D\x00\x00\x00u\x00\x00\x00o\x00\x00\x00w\x00\x00\x00\x06\x00\x00\x00j\x00\x00\x00i\x00\x00\x00\xb4\x00\x00\x00w\x00\x00\x00q\rtq\x0eb.')        
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ CUT HERE ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def my_agent(observation, configuration):
    global Gstate, Gprev_friend_move
    trials = 1000
    s = observation.step
    
    if (s == 0):
        Gstate = 0
        # make some dummy sizes
        Gprev_foe_move = -1
        Gprev_friend_move = -1
    if (s > 0):
        Gprev_foe_move = observation.lastOpponentAction
        
    Gstate, Gfriend_move = FSM_move_No_Numba(s, Gprev_friend_move, Gprev_foe_move, Gstate, F.state_choices, F.action_choices) 
    Gprev_friend_move = Gfriend_move
        
    return int(Gfriend_move)

A utility for printing two lines to get the FSM into the template. Run the code below. It prints out a couple of very long lines for reading the FSM in the form of a pickled string. Then copy the messy looking output and paste it into the template above, where it will look much like the two lines you will be replacing.

In [ ]:

import pickle
def print_results_F(F): # Spill a lot of gibberish to the screen for cutting and pasting
    a = pickle.dumps(F.action_choices)
    s = pickle.dumps(F.state_choices)
    print("F.action_choices = pickle.loads(", end = "")
    print( a, end = "")
    print(")")
    print("F.state_choices  = pickle.loads(", end = "")
    print( s, end = "")
    print(")")
 
print_results_F(pop[0])   #pop[0] holds the best individual found by the GA.

And that should do it. This agent isn't going to tear up the leaderboard but it's no longer as dumb as a box of rocks.